## ❗ Run this before going through steps, allows Jupyter to actually see installed packages ❗

In [11]:
!pip install ipykernel
!python3 -m ipykernel install --user --name=stylegan2env --display-name "Python (StyleGAN2)"

   
!jupyter kernelspec list

Defaulting to user installation because normal site-packages is not writeable
Installed kernelspec stylegan2env in /home/s25vargason1/.local/share/jupyter/kernels/stylegan2env
Available kernels:
  stylegan2env    /home/s25vargason1/.local/share/jupyter/kernels/stylegan2env
  venv            /home/s25vargason1/.local/share/jupyter/kernels/venv
  python3         /usr/local/share/jupyter/kernels/python3


In [12]:
import os

print("Working directory:", os.getcwd())
print("Files in cwd:", os.listdir("."))

Working directory: /home/s25vargason1/tess/stylegan2-ada-pytorch
Files in cwd: ['.git', '.github', '.gitignore', 'Dockerfile', 'LICENSE.txt', 'README.md', 'calc_metrics.py', 'dataset_tool.py', 'dnnlib', 'docker_run.sh', 'docs', 'generate.py', 'legacy.py', 'metrics', 'projector.py', 'style_mixing.py', 'torch_utils', 'train.py', 'training', 'datasets', '.ipynb_checkpoints', 'tess', 'logs', 'run.sh']


### 1. Refresh the page
### 2. Go to Kernel > Change Kernel
### 3. Select Python (StyleGAN2)
### 4. Start running through steps

## Step 1: Extract dataset images from zip file 

In [3]:
import zipfile
import os

# Just use relative paths directly
zip_path = "512_training/512images.zip"
extract_to = "images"

# Make sure output folder exists
os.makedirs(extract_to, exist_ok=True)

# Unpack zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    print("Extracting images...")
    zip_ref.extractall(extract_to)
    print("✅ Done! Extracted to:", extract_to)



Extracting images...
✅ Done! Extracted to: images


## Step 2: Clone the StyleGAN2-ADA repo from Github

In [ ]:
!git clone https://github.com/richvar/stylegan2-ada-pytorch.git
%cd stylegan2-ada-pytorch

# Print current directory to confirm
!pwd

# List contents to verify structure
!ls

# Step 3: Install needed requirements

In [ ]:
Check current directory

In [98]:
import os

%cd /home/s25vargason1/tess/512_training/images
print("Current directory:", os.getcwd())

/home/s25vargason1/tess/512_training/images
Current directory: /home/s25vargason1/tess/512_training/images


In [7]:
import os

# Check current working directory
cwd = os.getcwd()
expected = "stylegan2-ada-pytorch"

if expected not in os.path.basename(cwd):
    print(f"You're in '{cwd}', not inside the '{expected}' folder.")
    print("Use `%cd stylegan2-ada-pytorch` to navigate there before running this cell.")
else:
    print(f"In correct directory: {cwd}")

    # Manually install dependencies
    # !pip install torch==1.10.1 torchvision==0.11.2
    # !pip install click requests tqdm pyspng ninja imageio-ffmpeg scipy

    # Confirm install
    import torch
    import torchvision
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")

In correct directory: /home/s25vargason1/tess/stylegan2-ada-pytorch
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
PyTorch version: 1.10.1+cu102
CUDA available: False


In [ ]:
Create script to remove faces from photos and resize them to 5x5 

In [83]:
import zipfile
import os

zip_path = "/home/s25vargason1/tess/images/images.zip"
extract_to = "tess/512_training/images"


# Extract!
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"✅ Extracted to {extract_to}")


✅ Extracted to tess/512_training/images


## Step 4: Build StyleGAN2 dataset from image folder

In [111]:
import os

%cd /home/s25vargason1/tess/stylegan2-ada-pytorch
print("✅ Contents:", os.listdir())


/home/s25vargason1/tess/stylegan2-ada-pytorch
✅ Contents: ['.git', '.github', '.gitignore', 'Dockerfile', 'LICENSE.txt', 'README.md', 'calc_metrics.py', 'dataset_tool.py', 'dnnlib', 'docker_run.sh', 'docs', 'generate.py', 'legacy.py', 'metrics', 'projector.py', 'style_mixing.py', 'torch_utils', 'train.py', 'training', 'datasets', '.ipynb_checkpoints']


In [115]:
import os
import shutil
import subprocess

def convert_images_for_stylegan2ada_pytorch(
    source_dir="512_training/images",
    dest_dir="datasets/post-impressionist",
    force_overwrite=False
):
    source_dir = source_dir.strip()
    dest_dir = dest_dir.strip()

    if not os.path.exists(source_dir):
        print(f"❌ Source folder does not exist: {source_dir}")
        return
    if not os.listdir(source_dir):
        print(f"❌ Source folder is empty: {source_dir}")
        return

    if os.path.exists(dest_dir):
        if force_overwrite:
            print(f"🧹 Deleting existing folder: {dest_dir}")
            shutil.rmtree(dest_dir)
        else:
            print(f"❌ Destination exists and force_overwrite=False: {dest_dir}")
            return

    os.makedirs(dest_dir, exist_ok=True)

    # Build the command
    command = [
    "python3", "dataset_tool.py",  # just use the local file if you're in stylegan2-ada-pytorch
    f"--source={source_dir}",
    f"--dest={dest_dir}"
    ]

    print(f"🚀 Running: {' '.join(command)}")

    env = os.environ.copy()
    env["LC_ALL"] = "C.UTF-8"
    env["LANG"] = "C.UTF-8"

    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, env=env)
    stdout = result.stdout.decode("utf-8", errors="replace")
    stderr = result.stderr.decode("utf-8", errors="replace")


    if result.returncode == 0:
        print("✅ Conversion succeeded.")
        print(stdout)
    else:
        print("❌ Conversion failed.")
        print(stderr)


    # Confirm output
    if os.path.exists(dest_dir):
        shards = [d for d in os.listdir(dest_dir) if d.startswith("000")]
        json_file = os.path.join(dest_dir, "dataset.json")
        if shards and os.path.isfile(json_file):
            print(f"📦 Dataset ready with {len(shards)} shard folders.")
        else:
            print("⚠️ Output structure incomplete.")
    else:
        print("❌ Destination folder missing.")




In [116]:
convert_images_for_stylegan2ada_pytorch(
    source_dir="/home/s25vargason1/tess/512_training/images",
    dest_dir="tess/datasets/post-impressionist",
    force_overwrite=True
)


🧹 Deleting existing folder: tess/datasets/post-impressionist
🚀 Running: python3 dataset_tool.py --source=/home/s25vargason1/tess/512_training/images --dest=tess/datasets/post-impressionist
✅ Conversion succeeded.

📦 Dataset ready with 3 shard folders.


In [5]:
%cd /home/s25vargason1/tess/stylegan2-ada-pytorch

import os
print("📁 Current directory:", os.getcwd())
print("📄 Files:", os.listdir())




/home/s25vargason1/tess/stylegan2-ada-pytorch
📁 Current directory: /home/s25vargason1/tess/stylegan2-ada-pytorch
📄 Files: ['.git', '.github', '.gitignore', 'Dockerfile', 'LICENSE.txt', 'README.md', 'calc_metrics.py', 'dataset_tool.py', 'dnnlib', 'docker_run.sh', 'docs', 'generate.py', 'legacy.py', 'metrics', 'projector.py', 'style_mixing.py', 'torch_utils', 'train.py', 'training', 'datasets', '.ipynb_checkpoints', 'tess']


### DON'T USE THIS ## Step 5: Start training loop for the model

In [ ]:
import os
from datetime import datetime

!pip install --user psutil

# Define results and log output directory
log_dir = "./results"
log_file = os.path.join(log_dir, f"training_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")

# Create results directory if it doesn't exist
os.makedirs(log_dir, exist_ok=True)

# Start training
!export LC_ALL=C.UTF-8 && export LANG=C.UTF-8 && python3 train.py \
  --outdir=./results \
  --data=./datasets/post-impressionist \
  --gpus=1 \
  --snap=1 \
  --resume=ffhq512 \
  --cfg=auto \
  --kimg-per-tick=10 \
  --aug=ada | tee {log_file}

print(f"Training started in background. Logs will be saved to:\n {log_file}")

In [6]:
!conda info --envs



/bin/bash: conda: command not found


In [2]:
%cd /home/s25vargason1/tess/stylegan2-ada-pytorch



/home/s25vargason1/tess/stylegan2-ada-pytorch


In [4]:
import os
import shutil

# Confirm current directory
cwd = os.getcwd()
print("📁 Current directory:", cwd)

# Safety check before deleting
if os.path.basename(cwd) == "stylegan2-ada-pytorch" and os.path.isdir("tess"):
    print("🧹 Deleting 'tess' folder and all contents...")
    shutil.rmtree("tess")
    print("✅ 'tess' folder deleted.")
else:
    print("⚠️ Not in the correct directory or 'datasets' folder not found.")


📁 Current directory: /home/s25vargason1/tess/stylegan2-ada-pytorch
🧹 Deleting 'tess' folder and all contents...
✅ 'tess' folder deleted.
